Visualization of the filters of VGG16, via gradient ascent in input space.
This script can run on CPU in a few minutes.
Results example: http://i.imgur.com/4nj4KjN.jpg

In [1]:
from __future__ import print_function

import numpy as np
import time
from keras.preprocessing.image import save_img
from keras.applications import vgg16
from keras import backend as K

E:\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [2]:
# dimensions of the generated pictures for each filter.
img_width = 128
img_height = 128

# the name of the layer we want to visualize
# (see model definition at keras/applications/vgg16.py)
layer_name = 'block5_conv1'

# util function to convert a tensor into a valid image

In [3]:
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [4]:
# build the VGG16 network with ImageNet weights
model = vgg16.VGG16(weights='imagenet', include_top=False)
print('Model loaded.')

model.summary()

# this is the placeholder for the input images
input_img = model.input

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])

Model loaded.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0      

In [5]:
def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())

In [6]:
kept_filters = []
for filter_index in range(200):
    # we only scan through the first 200 filters,
    # but there are actually 512 of them
    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(20):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break

    # decode the resulting input image
    if loss_value > 0:
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))

Processing filter 0
Current loss value: 10.985356
Current loss value: 24.696487
Current loss value: 48.78093
Current loss value: 72.71332
Current loss value: 91.348236
Current loss value: 113.47569
Current loss value: 127.81845
Current loss value: 150.23633
Current loss value: 168.92828
Current loss value: 188.82043
Current loss value: 217.21326
Current loss value: 237.54446
Current loss value: 265.2977
Current loss value: 289.48276
Current loss value: 315.4129
Current loss value: 339.76984
Current loss value: 361.37665
Current loss value: 383.19385
Current loss value: 404.6997
Current loss value: 432.54715
Filter 0 processed in 3s
Processing filter 1
Current loss value: 0.0
Filter 1 processed in 0s
Processing filter 2
Current loss value: 10.249601
Current loss value: 30.657661
Current loss value: 58.049473
Current loss value: 99.123146
Current loss value: 129.01744
Current loss value: 163.71414
Current loss value: 192.13768
Current loss value: 223.78368
Current loss value: 249.52293
C

Current loss value: 498.9117
Current loss value: 533.46857
Current loss value: 567.34827
Current loss value: 598.5947
Current loss value: 631.69196
Filter 15 processed in 0s
Processing filter 16
Current loss value: 0.0
Filter 16 processed in 0s
Processing filter 17
Current loss value: 0.5654445
Current loss value: 8.581477
Current loss value: 26.141453
Current loss value: 58.6004
Current loss value: 96.00749
Current loss value: 141.90132
Current loss value: 176.16823
Current loss value: 224.31827
Current loss value: 285.60913
Current loss value: 342.7166
Current loss value: 402.64526
Current loss value: 471.80743
Current loss value: 548.2905
Current loss value: 608.7152
Current loss value: 669.5514
Current loss value: 734.3824
Current loss value: 796.40936
Current loss value: 852.31616
Current loss value: 914.73035
Current loss value: 969.4758
Filter 17 processed in 0s
Processing filter 18
Current loss value: 38.51999
Current loss value: 194.05713
Current loss value: 325.94766
Current 

Current loss value: 0.0
Filter 33 processed in 0s
Processing filter 34
Current loss value: 4.069995
Current loss value: 14.45782
Current loss value: 45.171986
Current loss value: 93.25922
Current loss value: 159.39969
Current loss value: 243.44502
Current loss value: 327.7111
Current loss value: 397.0424
Current loss value: 470.04022
Current loss value: 530.052
Current loss value: 585.03613
Current loss value: 645.10315
Current loss value: 693.6909
Current loss value: 753.1238
Current loss value: 795.9302
Current loss value: 846.68335
Current loss value: 899.97253
Current loss value: 948.57605
Current loss value: 998.4302
Current loss value: 1045.9502
Filter 34 processed in 0s
Processing filter 35
Current loss value: 0.0
Filter 35 processed in 0s
Processing filter 36
Current loss value: 5.6994157
Current loss value: 29.293888
Current loss value: 82.616234
Current loss value: 154.78006
Current loss value: 237.42247
Current loss value: 336.7859
Current loss value: 417.51923
Current loss 

Current loss value: 1047.4836
Current loss value: 1106.6814
Current loss value: 1155.8202
Current loss value: 1206.5297
Current loss value: 1256.9705
Current loss value: 1310.4019
Filter 48 processed in 0s
Processing filter 49
Current loss value: 0.6315441
Current loss value: 6.3517523
Current loss value: 16.661367
Current loss value: 65.34988
Current loss value: 111.33482
Current loss value: 182.7511
Current loss value: 269.706
Current loss value: 356.38736
Current loss value: 451.45547
Current loss value: 527.30945
Current loss value: 603.0155
Current loss value: 671.79987
Current loss value: 740.5935
Current loss value: 813.33923
Current loss value: 884.10547
Current loss value: 949.9389
Current loss value: 1019.7108
Current loss value: 1080.3427
Current loss value: 1148.9688
Current loss value: 1209.7295
Filter 49 processed in 0s
Processing filter 50
Current loss value: 1.5405302
Current loss value: 5.632217
Current loss value: 11.947122
Current loss value: 19.140469
Current loss v

Current loss value: 311.31284
Current loss value: 363.01608
Current loss value: 404.9668
Current loss value: 451.0798
Current loss value: 491.31375
Current loss value: 537.50574
Current loss value: 578.9253
Current loss value: 623.44165
Current loss value: 660.3784
Current loss value: 698.0132
Current loss value: 736.7707
Current loss value: 773.2983
Current loss value: 808.4536
Filter 65 processed in 0s
Processing filter 66
Current loss value: 10.99331
Current loss value: 17.512209
Current loss value: 32.97331
Current loss value: 58.770603
Current loss value: 100.1265
Current loss value: 138.00703
Current loss value: 182.19232
Current loss value: 229.84119
Current loss value: 272.4145
Current loss value: 321.7392
Current loss value: 363.496
Current loss value: 416.593
Current loss value: 454.09583
Current loss value: 507.11255
Current loss value: 552.3518
Current loss value: 611.61755
Current loss value: 657.17224
Current loss value: 714.8284
Current loss value: 759.78906
Current loss

...

Current loss value: 0.0
Filter 124 processed in 0s
Processing filter 125
Current loss value: 1.0859306
Current loss value: 10.75497
Current loss value: 29.117056
Current loss value: 65.96472
Current loss value: 100.87767
Current loss value: 139.34924
Current loss value: 179.3774
Current loss value: 221.86017
Current loss value: 258.7722
Current loss value: 298.7157
Current loss value: 344.8331
Current loss value: 389.18118
Current loss value: 433.6164
Current loss value: 482.0462
Current loss value: 536.02386
Current loss value: 596.7216
Current loss value: 660.5144
Current loss value: 721.32855
Current loss value: 784.40076
Current loss value: 848.14404
Filter 125 processed in 1s
Processing filter 126
Current loss value: 27.703144
Current loss value: 69.84613
Current loss value: 143.11374
Current loss value: 211.22345
Current loss value: 279.9571
Current loss value: 344.8963
Current loss value: 410.1678
Current loss value: 470.5077
Current loss value: 527.8556
Current loss value: 584.

Current loss value: 668.81433
Current loss value: 713.53845
Current loss value: 758.4469
Current loss value: 806.1001
Current loss value: 848.83826
Current loss value: 891.6487
Filter 139 processed in 1s
Processing filter 140
Current loss value: 11.522076
Current loss value: 41.722233
Current loss value: 81.33031
Current loss value: 118.35168
Current loss value: 157.27069
Current loss value: 197.41345
Current loss value: 232.04425
Current loss value: 264.73975
Current loss value: 303.66248
Current loss value: 329.99176
Current loss value: 371.52567
Current loss value: 408.16373
Current loss value: 441.79428
Current loss value: 485.73535
Current loss value: 524.1106
Current loss value: 569.21246
Current loss value: 603.8799
Current loss value: 646.83484
Current loss value: 681.7904
Current loss value: 722.2197
Filter 140 processed in 1s
Processing filter 141
Current loss value: 3.031334
Current loss value: 27.38876
Current loss value: 66.394714
Current loss value: 116.45792
Current loss

Current loss value: 234.7979
Current loss value: 277.67917
Current loss value: 325.28046
Current loss value: 374.8806
Current loss value: 424.62207
Current loss value: 466.3062
Current loss value: 510.85846
Current loss value: 554.78687
Current loss value: 603.326
Current loss value: 639.1503
Current loss value: 689.2012
Current loss value: 726.42926
Filter 153 processed in 1s
Processing filter 154
Current loss value: 18.157557
Current loss value: 72.22722
Current loss value: 147.98288
Current loss value: 200.14558
Current loss value: 257.26685
Current loss value: 317.58813
Current loss value: 383.71768
Current loss value: 435.09094
Current loss value: 493.9348
Current loss value: 538.2639
Current loss value: 595.472
Current loss value: 640.05743
Current loss value: 692.68646
Current loss value: 730.2742
Current loss value: 783.2232
Current loss value: 828.03174
Current loss value: 871.5308
Current loss value: 917.3371
Current loss value: 956.9053
Current loss value: 1006.7102
Filter 1

Current loss value: 0.0
Filter 169 processed in 0s
Processing filter 170
Current loss value: 17.12256
Current loss value: 35.669064
Current loss value: 74.004425
Current loss value: 111.30423
Current loss value: 153.47751
Current loss value: 195.51337
Current loss value: 253.96545
Current loss value: 300.49
Current loss value: 344.9856
Current loss value: 390.2439
Current loss value: 437.17737
Current loss value: 481.80154
Current loss value: 523.1898
Current loss value: 571.42664
Current loss value: 616.8954
Current loss value: 666.04266
Current loss value: 705.64923
Current loss value: 754.79175
Current loss value: 804.2725
Current loss value: 851.5703
Filter 170 processed in 1s
Processing filter 171
Current loss value: 44.149902
Current loss value: 107.610855
Current loss value: 178.1543
Current loss value: 253.9693
Current loss value: 326.98947
Current loss value: 391.02612
Current loss value: 446.50812
Current loss value: 492.1383
Current loss value: 555.42
Current loss value: 607

Current loss value: 299.96057
Current loss value: 346.0475
Current loss value: 397.14957
Current loss value: 439.7229
Current loss value: 489.81842
Current loss value: 532.99414
Current loss value: 578.07764
Current loss value: 622.3672
Current loss value: 663.83234
Current loss value: 705.21643
Current loss value: 743.67535
Current loss value: 786.05554
Filter 186 processed in 1s
Processing filter 187
Current loss value: 12.19409
Current loss value: 80.456345
Current loss value: 204.28896
Current loss value: 327.69785
Current loss value: 428.60785
Current loss value: 529.1338
Current loss value: 621.02124
Current loss value: 711.38525
Current loss value: 790.5343
Current loss value: 866.1033
Current loss value: 941.26746
Current loss value: 1018.6859
Current loss value: 1092.543
Current loss value: 1164.0776
Current loss value: 1236.3699
Current loss value: 1305.0171
Current loss value: 1377.2168
Current loss value: 1443.404
Current loss value: 1510.8813
Current loss value: 1580.1365


In [7]:
# we will stich the best 64 filters on a 8 x 8 grid.
n = 8

# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 64 filters.
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

# build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

In [9]:
# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        width_margin = (img_width + margin) * i
        height_margin = (img_height + margin) * j
        stitched_filters[
            width_margin: width_margin + img_width,
            height_margin: height_margin + img_height, :] = img

# save the result to disk
save_img('stitched_filters_%dx%d.png' % (n, n), stitched_filters)

<img style="float: center;" src="stitched_filters_8x8.png">